# **Android's Forge Colab**
# Forked from (cagliostro-forge-colab)
Rise from the ashes, reborn and empowered by [lllyasviel/stable-diffusion-webui-forge](https://github.com/lllyasviel/stable-diffusion-webui-forge)

Changelog:
- Added Adetailer
- Added CivitAI Browser extension (You can now download loras directly in the WebUI)
- Added 4xAnimeSharp as ESRGAN.
- Added Anzhc Yolov5 adetailer models.
- Added Field to input CivitAI token to download civitai models.
- Added SD Forge Couple extension.
- Added Flux1D and Schnell Lora for Flux1D
- Added Option for including a "Custom Lora Directory" where you can load a directory of loras from google drive.
- Added New Field for inputting a custom upscaler, along with 4xUltraSharp as a default model.
- Added Yolo adetailer models for Hand and Feet.
- Added Hassaku XL (Illustrious) as a new default model.



In [ ]:
# @title ## (1) **Install Environment**
import sys
import subprocess
import os
import time
import json
import shutil
import random
import string
from pathlib import Path
from tqdm import tqdm
from pydantic import BaseModel
import zipfile

requirements_file = """
GitPython==3.1.32
accelerate
blendmodes==2022
clean-fid==0.1.35
diskcache==5.6.3
einops
facexlib==0.3.0
fastapi==0.104.1
gradio==4.40.0
httpcore==0.15
inflection==0.5.1
jsonmerge==1.8.0
kornia==0.6.7
lark==1.1.2
numpy
omegaconf==2.2.3
open-clip-torch==2.20.0
piexif==1.1.3
protobuf
psutil==5.9.5
pytorch_lightning==1.9.4
resize-right==0.0.2
safetensors
scikit-image
spandrel==0.3.4
spandrel-extra-arches==0.1.1
tomesd==0.1.3
torchdiffeq==0.2.3
torchsde==0.2.6
transformers==4.46.1
httpx==0.24.1
pillow-avif-plugin==1.4.3
diffusers==0.31.0
gradio_rangeslider==0.0.6
gradio_imageslider==0.0.20
loadimg==0.1.2
tqdm==4.66.1
peft
pydantic==2.8.2
huggingface-hub==0.26.2
peft==0.12.0
pillow
"""

python_version  = ".".join(sys.version.split(".")[:2])
python_path     = Path(f"/usr/local/lib/python{python_version}/dist-packages/")
colablib_path   = python_path / "colablib"
if not colablib_path.exists():
    subprocess.run(['pip', 'install', '--upgrade', 'git+https://github.com/Linaqruf/colablib'], check=True)

from colablib.colored_print import cprint, print_line
from colablib.utils import py_utils, package_utils, config_utils
from colablib.sd_models.downloader import aria2_download, download
from colablib.utils.git_utils import update_repo, reset_repo, validate_repo, batch_update
from colablib.utils.py_utils import get_filename

################################
# COLAB ARGUMENTS GOES HERE
################################

# It ain't much, but it's honest work.
class CustomDirs(BaseModel):
    url: str
    dst: str

# @markdown ### **Drive Config**
mount_drive          = True  # @param {type: 'boolean'}
output_drive_folder  = "android-colab-forge"  # @param {type: 'string'}

# @markdown ### **Repo Config**
update_webui         = True  # @param {type: 'boolean'}
update_extensions    = True  # @param {type: 'boolean'}
commit_hash          = ""  # @param {type: 'string'}

# @markdown ### **Download Config**
# @markdown > Check only the options you need
animagine_xl_3_1     = False  # @param {type: 'boolean'}
rae_diffusion_xl_v2  = False  # @param {type: 'boolean'}
kivotos_xl_v2_0      = False  # @param {type: 'boolean'}
urangdiffusion_2_0   = False  # @param {type: 'boolean'}

# @markdown - The following models require Civitai token
wai_nsfw_illustrious_sdxl = False  # @param {type: 'boolean'}
illustrious_xl_smoothft = False  # @param {type: 'boolean'}
madly_mix_ver_nightnoob = False  # @param {type: 'boolean'}
hassaku_xl_illustrious = True  # @param {type: 'boolean'}

# @markdown - You will need to set Diffusion in Low Bits to bnb nf4 (fp16 Lora) for the following model
flux1_dev_bnb_nf4    = False  # @param {type: 'boolean'}

# @markdown ### **Miscellaneous Loras**
# @markdown > Check only the options you need (Also need Civitai token)
schnell_lora_for_flux1dev = False  # @param {type: 'boolean'}

# @markdown > **Note:**
# @markdown - For multiple URLs, use comma separation (e.g. `url1, url2, url3`)
# @markdown - Forge supports FLUX, SD, and SDXL
# @markdown - **Highly Recommended:** Use Hugging Face links whenever possible (Click the "Copy Download Link" button on the model page)
# @markdown - **For civitai models**: Right click the download button and paste that link, but for some models you will need your civitai token, so include it below.
civitai_token        = ""  # @param {type: 'string'}
custom_model_url     = ""  # @param {'type': 'string'}
custom_vae_url       = "https://huggingface.co/madebyollin/sdxl-vae-fp16-fix/resolve/main/sdxl.vae.safetensors"  # @param {'type': 'string'}
custom_lora_url      = ""  # @param {'type': 'string'}
custom_upscaler_url   = ""  # @param {'type': 'string'}

# @markdown ### **Custom Extensions**
# @markdown > Enter the GitHub URLs for the extensions you want to install, separated by commas.
custom_extensions_url = ""  # @param {type: 'string'}

# @markdown ### **If you have a lora folder in your drive, you can set it here, and it will add the loras to the lora folder**
# @markdown > **Note:** This will not download the loras, it will just add them to the lora folder
# @markdown - load_recursively: If checked, it will load all the loras in the folder and subfolders
custom_lora_dir     = ""  # @param {'type': 'string'}
load_recursively    = True  # @param {type: 'boolean'}

# @markdown ### **Tunnel Config**
# @markdown > Default to `--share` until `ngrok_token` is not `None`
ngrok_token          = ""  # @param {type: 'string'}
ngrok_region         = "us"  # @param ["us", "eu", "au", "ap", "sa", "jp", "in"]

# @markdown ### **UI/UX Config**
gradio_theme         = "remilia/Ghostly"  # @param ["Default", "gradio/base", "gradio/glass", "gradio/monochrome", "gradio/seafoam", "gradio/soft", "gradio/dracula_test", "abidlabs/dracula_test", "abidlabs/Lime", "abidlabs/pakistan", "Ama434/neutral-barlow", "dawood/microsoft_windows", "finlaymacklon/smooth_slate", "Franklisi/darkmode", "freddyaboulton/dracula_revamped", "freddyaboulton/test-blue", "gstaff/xkcd", "Insuz/Mocha", "Insuz/SimpleIndigo", "JohnSmith9982/small_and_pretty", "nota-ai/theme", "nuttea/Softblue", "ParityError/Anime", "reilnuud/polite", "remilia/Ghostly", "rottenlittlecreature/Moon_Goblin", "step-3-profit/Midnight-Deep", "Taithrah/Minimal", "ysharma/huggingface", "ysharma/steampunk", "NoCrypt/miku"]
# @markdown Set `use_preset` for using default prompt, resolution, sampler, and other settings
use_presets          = True  # @param {type: 'boolean'}

# @markdown ### **Launch Arguments**
use_gradio_auth      = False  # @param {type: 'boolean'}
auto_select_model    = False  # @param {type: 'boolean'}
auto_select_vae      = True  # @param {type: 'boolean'}
additional_arguments = "--lowram --theme dark --no-half-vae --opt-sdp-attention"  # @param {type: 'string'}

################################
# GLOBAL VARIABLES GOES HERE
################################

# GRADIO AUTH
user      = "android"
password  = "".join(random.choices(string.ascii_letters + string.digits, k=6))

# ROOT DIR
root_dir        = Path("/content")
drive_dir       = root_dir / "drive" / "MyDrive"
repo_dir        = root_dir / "stable-diffusion-webui-forge"
tmp_dir         = root_dir / "tmp"

models_dir      = repo_dir / "models"
extensions_dir  = repo_dir / "extensions"
ckpt_dir        = models_dir / "Stable-diffusion"
vae_dir         = models_dir / "VAE"
lora_dir        = models_dir / "Lora"
output_subdir   = ["txt2img-samples", "img2img-samples", "extras-samples", "txt2img-grids", "img2img-grids"]

config_file_path    = repo_dir / "config.json"
ui_config_file_path = repo_dir / "ui-config.json"

package_url = [
    "https://huggingface.co/Linaqruf/fast-repo/resolve/main/webui-forge.tar.lz4",
    "https://huggingface.co/Linaqruf/fast-repo/resolve/main/webui-forge-deps.tar.lz4"
]

custom_dirs = {
    "model" : CustomDirs(url=custom_model_url, dst=str(ckpt_dir)),
    "vae"   : CustomDirs(url=custom_vae_url, dst=str(vae_dir)),
    "lora"  : CustomDirs(url=custom_lora_url, dst=str(lora_dir)),
}

default_model_urls = {
    "animagine_xl_3_1"         : "https://huggingface.co/cagliostrolab/animagine-xl-3.1/resolve/main/animagine-xl-3.1.safetensors",
    "rae_diffusion_xl_v2"      : "https://huggingface.co/Raelina/Rae-Diffusion-XL-V2/resolve/main/RaeDiffusion-XL-v2.safetensors",
    "kivotos_xl_v2_0"          : "https://huggingface.co/yodayo-ai/kivotos-xl-2.0/resolve/main/kivotos-xl-2.0.safetensors",
    "urangdiffusion_2_0"       : "https://huggingface.co/kayfahaarukku/UrangDiffusion-2.0/resolve/main/UrangDiffusion-2.0.safetensors",
    "wai_nsfw_illustrious_sdxl": "https://civitai.com/api/download/models/1490781?type=Model&format=SafeTensor&size=pruned&fp=fp16",
    "illustrious_xl_smoothft"  : "https://civitai.com/api/download/models/1015877?type=Model&format=SafeTensor&size=pruned&fp=fp16",
    "madly_mix_ver_nightnoob"  : "https://civitai.com/api/download/models/1202045?type=Model&format=SafeTensor&size=full&fp=fp16",
    "hassaku_xl_illustrious"   : "https://civitai.com/api/download/models/1240288?type=Model&format=SafeTensor&size=pruned&fp=bf16",
    "flux1_dev_bnb_nf4"        : "https://huggingface.co/lllyasviel/flux1-dev-bnb-nf4/blob/main/flux1-dev-bnb-nf4-v2.safetensors"
}

default_lora_urls = {
    "schnell_lora_for_flux1dev": "https://civitai.com/api/download/models/759853?type=Model&format=SafeTensor"
}

################################
# HELPER FUNCTIONS STARTS HERE
################################

def mount_drive_function(directory):
    output_dir = repo_dir / "outputs"

    if mount_drive:
        print_line(80, color="green")
        if not directory.exists():
            from google.colab import drive

            cprint("Mounting google drive...", color="green", reset=False)
            drive.mount(str(directory.parent))
        output_dir = directory / output_drive_folder
        cprint("Set default output path to:", output_dir, color="green")

    return output_dir

def setup_directories():
    for dir in [ckpt_dir, vae_dir, lora_dir]:
        dir.mkdir(parents=True, exist_ok=True)

def pre_download(dir, urls, desc, overwrite=False):
    ffmpy_path = python_path / "ffmpy-0.3.0.dist-info"

    for url in tqdm(urls, desc=desc):
        filename = Path(url).name
        aria2_download(dir, filename, url, quiet=True)
        if filename == "webui-forge-deps.tar.lz4":
            package_utils.extract_package(filename, python_path, overwrite=True)
        else:
            package_utils.extract_package(filename, "/", overwrite=overwrite)
        os.remove(dir / filename)

    subprocess.run(["rm", "-rf", str(ffmpy_path)])
    subprocess.run(["pip", "install", "--force-reinstall", "ffmpy"], check=True)

def install_dependencies():
    ubuntu_deps = ["aria2", "lz4"]
    cprint("Installing ubuntu dependencies", color="green")
    subprocess.run(["apt", "install", "-y"] + ubuntu_deps, check=True)

def install_webui(repo_dir, desc):
    if not repo_dir.exists():
        pre_download(root_dir, package_url, desc, overwrite=False)

    else:
        cprint("Stable Diffusion Web UI forge already installed, skipping...", color="green")

def configure_output_path(config_path, output_dir, output_subdir):
    try:
        config = config_utils.read_config(str(config_path))
    except (FileNotFoundError, json.JSONDecodeError):
        config = {}

    config_updates = {
        f"outdir_{subdir.split('-')[0]}_{'_'.join(subdir.split('-')[1:])}": str(output_dir / subdir)
        for subdir in output_subdir
    }

    config.update(config_updates)

    config_path.parent.mkdir(parents=True, exist_ok=True)

    config_utils.write_config(str(config_path), config)

    for dir in output_subdir:
        (output_dir / dir).mkdir(parents=True, exist_ok=True)

def prepare_environment():
    cprint("Preparing environment...", color="green")
    os.environ['PYTORCH_CUDA_ALLOC_CONF']   = "garbage_collection_threshold:0.9,max_split_size_mb:512"
    os.environ["TF_CPP_MIN_LOG_LEVEL"]      = "3"
    os.environ["PYTHONWARNINGS"]            = "ignore"

def custom_download(custom_dirs):
    filtered_urls = filter_dict_items(default_model_urls)
    filtered_lora_urls = filter_dict_items(default_lora_urls)

    for key, value in custom_dirs.items():
        urls = value.url.split(",")
        dst = value.dst

        if key == "model":
            urls.extend(filtered_urls)

        if key == "lora":
            urls.extend(filtered_lora_urls)

        if urls[0]:
            print_line(80, color="green")
            cprint(f" [-] Downloading Custom {key}...", color="flat_yellow")

        for url in urls:
            if "civitai.com" in url:
                if civitai_token:
                    url = f"{url}&token={civitai_token}"
                else:
                    cprint(f"Civitai token is required for this model {url}, skipping...", color="red")
                    continue
            url = url.strip()
            if url != "":
                print_line(80, color="green")
                if "|" in url:
                    url, filename = map(str.strip, url.split("|"))
                    if not filename.endswith((".safetensors", ".ckpt", ".pt", "pth")):
                        filename = filename + Path(get_filename(url)).suffix
                else:
                    filename = get_filename(url)

                download(url=url, filename=filename, dst=dst, quiet=False)

def filter_dict_items(dict_items):
    result_list = []
    for key, url in dict_items.items():
        if globals().get(key):
            result_list.append(url)
    return result_list

def auto_select_file(target_dir, config_key, file_types):
    valid_files = [f for f in os.listdir(target_dir) if f.endswith(file_types)]
    if valid_files:
        file_path = random.choice(valid_files)

        if Path(target_dir).joinpath(file_path).exists():
            config = config_utils.read_config(str(config_file_path))
            config[config_key] = file_path
            config_utils.write_config(str(config_file_path), config)
        return file_path
    else:
        return None

def ui_config_presets():
    preset_prompt = "masterpiece, best quality, very aesthetic, absurdres"
    preset_negative_prompt = "nsfw, lowres, (bad), text, error, fewer, extra, missing, worst quality, jpeg artifacts, low quality, watermark, unfinished, displeasing, oldest, early, chromatic aberration, signature, extra digits, artistic error, username, scan, [abstract]"

    return {
        "txt2img/Prompt/value"              : preset_prompt,
        "txt2img/Negative prompt/value"     : preset_negative_prompt,
        "img2img/Prompt/value"              : preset_prompt,
        "img2img/Negative prompt/value"     : preset_negative_prompt,
        "customscript/sampler.py/txt2img/Sampling method/value" : "Euler a",
        "customscript/sampler.py/txt2img/Sampling steps/value"  : 28,
        "customscript/sampler.py/txt2img/Scheduler/value"       : "Automatic",
    }

def ui_config_settings(ui_config_file: str):
    config = config_utils.read_config(str(ui_config_file))
    preset_config = ui_config_presets()

    for key, value in preset_config.items():
        config[key] = value

    config_utils.write_config(str(ui_config_file), config)

def general_config_presets(config_file: str, lora_dir: str, use_presets: bool, ui_config_file: str):
    config = config_utils.read_config(str(config_file))

    config.update({
        "CLIP_stop_at_last_layers"      : 2,
        "show_progress_every_n_steps"   : 10,
        "show_progressbar"              : True,
        "samples_filename_pattern"      : "[model_name]_[seed]",
        "show_progress_type"            : "Approx NN",
        "live_preview_content"          : "Prompt",
        "forge_preset"                  : "xl",
        "xl_t2i_width"                  : 832,
        "xl_t2i_height"                 : 1216,
        "xl_t2i_cfg"                    : 5.5,
        "xl_t2i_hr_cfg"                 : 7,
        "xl_t2i_sampler"                : "Euler a",
        "xl_t2i_scheduler"              : "Automatic",
        "gradio_theme"                  : gradio_theme,
    })

    config_utils.write_config(str(config_file), config)

    if use_presets:
        ui_config_settings(ui_config_file)

def is_valid(target_dir, file_types):
    return any(f.endswith(file_types) for f in os.listdir(target_dir))

def parse_args(config):
    args = []
    for k, v in config.items():
        if k.startswith("_"):
            args.append(f'"{v}"')
        elif isinstance(v, str):
            args.append(f'--{k}="{v}"')
        elif isinstance(v, bool) and v:
            args.append(f"--{k}")
        elif isinstance(v, (float, int)) and not isinstance(v, bool):
            args.append(f"--{k}={v}")
    return " ".join(args)

def main():
    global output_dir, auto_select_model, auto_select_vae, final_args

    ################################
    # MAIN EXECUTION
    ################################

    os.chdir(root_dir)
    start_time = time.time()
    output_dir = mount_drive_function(drive_dir)

    gpu_info    = py_utils.get_gpu_info(get_gpu_name=True)
    python_info = py_utils.get_python_version()
    torch_info  = py_utils.get_torch_version()

    print_line(80, color="green")
    cprint(f" [-] Current GPU: {gpu_info}", color="flat_yellow")
    cprint(f" [-] Python {python_info}", color="flat_yellow")
    cprint(f" [-] Torch {torch_info}", color="flat_yellow")
    print_line(80, color="green")

    try:
        install_dependencies()

        print_line(80, color="green")
        install_webui(repo_dir, cprint("Unpacking Web UI forge", color="green", tqdm_desc=True))
        prepare_environment()

        configure_output_path(config_file_path, output_dir, output_subdir)

        print_line(80, color="green")
        if update_webui and not commit_hash:
            update_repo(cwd=repo_dir, args="-X theirs --rebase --autostash")
        elif commit_hash:
            reset_repo(repo_dir, commit_hash)

        setup_directories()

        repo_name, current_commit_hash, current_branch = validate_repo(repo_dir)
        cprint(f"Using '{repo_name}' repository...", color="green")
        cprint(f"Branch: {current_branch}, Commit hash: {current_commit_hash}", color="green")

        if update_extensions:
            print_line(80, color="green")
            batch_update(fetch=True, directory=extensions_dir, desc=cprint("Updating extensions", color="green", tqdm_desc=True))


        # addding adetailer
        print_line(80, color="green")
        cprint("Installing adetailer...", color="green")
        if not Path(extensions_dir / "adetailer").exists():
            !git clone https://github.com/Bing-su/adetailer.git {extensions_dir / "adetailer"}
        else:
            cprint("Adetailer already installed, skipping...", color="green")

        # Added Anzhc Yolo models to adetailer (by placing it in models/adetailer)
        print_line(80, color="green")
        cprint("Installing Adetailer/Yolo models...", color="green")
        anzhc_models = ["https://huggingface.co/Anzhc/Anzhcs_YOLOs/blob/main/Anzhc%20Breasts%20Seg%20v1%201024m.pt",
                        "https://huggingface.co/Anzhc/Anzhcs_YOLOs/blob/main/Anzhc%20Eyes%20-seg-hd.pt",
                        "https://huggingface.co/Anzhc/Anzhcs_YOLOs/blob/main/Anzhc%20Face%20-seg.pt",
                        "https://huggingface.co/Anzhc/Anzhcs_YOLOs/blob/main/Anzhc%20Face%20seg%201024%20v2%20y8n.pt",
                        "https://huggingface.co/Anzhc/Anzhcs_YOLOs/blob/main/Anzhc%20HeadHair%20seg%20y8m.pt",
                        "https://huggingface.co/Anzhc/Anzhcs_YOLOs/blob/main/Anzhc%20Manga%20Panels%20-seg.pt",
                        "https://huggingface.co/Anzhc/Anzhcs_YOLOs/blob/main/Anzhcs%20ManFace%20v02%201024%20y8n.pt",
                        "https://huggingface.co/Anzhc/Anzhcs_YOLOs/blob/main/Anzhcs%20WomanFace%20v05%201024%20y8n.pt",
                        "https://huggingface.co/Bingsu/adetailer/resolve/main/hand_yolov9c.pt",
                        "https://huggingface.co/MonetEinsley/ADetailer_CM/resolve/main/foot_yolov8x_v2.pt"]

        for url in anzhc_models:
            # if the file is not found, download it
            if not Path(models_dir / "adetailer" / get_filename(url)).exists():
                download(url=url, filename=get_filename(url), dst=str(models_dir / "adetailer"), quiet=False)
            else:
                cprint(f"Model {get_filename(url)} already exists, skipping...", color="green")

        # Adding Upscalers to models/ESRGAN
        print_line(80, color="green")
        cprint("Installing ESRGAN models...", color="green")
        esrgan_models = ["https://huggingface.co/Kim2091/AnimeSharp/blob/main/4x-AnimeSharp.safetensors",
                        "https://huggingface.co/Kim2091/UltraSharp/resolve/main/4x-UltraSharp.pth?download=true"
                         ]

        esrgan_models.extend([i for i in esrgan_models if i])

        for url in esrgan_models:
            if not Path(models_dir / "ESRGAN" / get_filename(url)).exists():
                download(url=url, filename=get_filename(url), dst=str(models_dir / "ESRGAN"), quiet=False)
            else:
                cprint(f"Model {get_filename(url)} already exists, skipping...", color="green")

        # adding civitai browser
        print_line(80, color="green")
        cprint("Installing civitai browser...", color="green")
        civitai_browser_dir = extensions_dir / "sd-civitai-browser-plus"

        if not civitai_browser_dir.exists():
            download_url = "https://civitai.com/api/download/models/964702?type=Archive&format=Other"
            zip_path = tmp_dir / "civitai_browser.zip"
            # Download the zip file
            download(url=download_url, filename="civitai_browser.zip", dst=str(tmp_dir), quiet=False)
            # Unzip to extensions directory
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(extensions_dir)
            # Clean up
            os.remove(zip_path)
        else:
            cprint("Civitai browser already installed, skipping...", color="green")

        # adding SD Forge Couple
        print_line(80, color="green")
        cprint("Installing SD Forge Couple...", color="green")
        sd_forge_couple_dir = extensions_dir / "sd-forge-couple"

        if not sd_forge_couple_dir.exists():
            !git clone https://github.com/Haoming02/sd-forge-couple {sd_forge_couple_dir}
        else:
            cprint("SD Forge Couple already installed, skipping...", color="green")

        # installing custom extensions
        if custom_extensions_url:
            print_line(80, color="green")
            cprint("Installing Custom Extensions...", color="green")
            for url in custom_extensions_url.split(","):
                url = url.strip()
                if not url: continue
                repo_name = url.split("/")[-1].replace(".git", "")
                ext_path = extensions_dir / repo_name

                if not ext_path.exists():
                    cprint(f"Cloning {repo_name}...", color="green")
                    try:
                        subprocess.run(["git", "clone", url, str(ext_path)], check=True)
                    except subprocess.CalledProcessError:
                        cprint(f"Failed to clone {repo_name}", color="red")
                else:
                    cprint(f"Extension {repo_name} already installed, skipping...", color="green")

        elapsed_time = py_utils.calculate_elapsed_time(start_time)
        print_line(80, color="green")

        # adding loras from custom lora directory
        print_line(80, color="green")
        cprint(f"Installing custom loras from '{custom_lora_dir}'", color="green")
        lora_count = 0
        if custom_lora_dir:
            if load_recursively:
                custom_lora_path = Path(custom_lora_dir)
                for root, dirs, files in os.walk(custom_lora_path):
                    for file in files:
                        if file.endswith((".ckpt", ".safetensors", ".pt")):
                            src_path = Path(root) / file
                            dst_path = lora_dir / file

                            # check if the file already exists
                            if dst_path.exists():
                                # if the file exists, skip it
                                cprint(f"File {dst_path} already exists, skipping...", color="yellow")
                                continue
                            else:
                                # if the file doesn't exist, copy it
                                cprint(f"Adding {file}", color="green")

                            shutil.copy(src_path, dst_path)
                            lora_count += 1
            else:
                custom_lora_path = Path(custom_lora_dir)
                if not lora_dir.exists():
                    cprint(f"Custom lora directory {lora_dir} does not exist, skipping...", color="red")
                    return
                for file in os.listdir(custom_lora_path):
                    if file.endswith((".ckpt", ".safetensors", ".pt")):
                        src_path = custom_lora_path / file
                        dst_path = lora_dir / file

                        # check if the file already exists
                        if dst_path.exists():
                            # if the file exists, skip it
                            cprint(f"File {dst_path} already exists, skipping...", color="yellow")
                            continue
                        else:
                            # if the file doesn't exist, copy it
                            cprint(f"Adding {file}", color="green")

                        shutil.copy(src_path, dst_path)
                        lora_count += 1

        if lora_count == 0:
            cprint("No loras found in the custom lora directory, skipping...", color="red")
        else:
            cprint(f"Found {lora_count} loras in the custom lora directory.", color="green")
        cprint("Installing custom loras finished.", color="green")

        cprint(f"Finished installation. Took {elapsed_time}.", color="flat_yellow")
    except Exception as e:
        cprint(f"An error occurred: {str(e)}", color="red")
        print_line(80, color="red")
        cprint("Setup failed. Please check the error message above and try again.", color="red")
        print_line(80, color="red")
        return

    start_time = time.time()

    custom_download(custom_dirs)

    elapsed_time = py_utils.calculate_elapsed_time(start_time)
    print_line(80, color="green")
    cprint(f"Download finished. Took {elapsed_time}.", color="flat_yellow")
    print_line(80, color="green")
    cprint(f"Launching '{repo_name}'", color="flat_yellow")
    print_line(80, color="green")

    if not is_valid(ckpt_dir, ('.ckpt', '.safetensors')):
        cprint(f"No checkpoints were found in the directory '{ckpt_dir}'.", color="yellow")
        url = "https://huggingface.co/cagliostrolab/animagine-xl-3.1/blob/main/animagine-xl-3.1.safetensors"
        filename = get_filename(url)
        aria2_download(url=url, download_dir=ckpt_dir, filename=filename)
        print_line(80, color="green")
        auto_select_model = True

    if not is_valid(vae_dir, ('.vae.pt', '.vae.safetensors', '.pt', '.ckpt')):
        cprint(f"No VAEs were found in the directory '{vae_dir}'.", color="yellow")
        url = "https://huggingface.co/madebyollin/sdxl-vae-fp16-fix/blob/main/sdxl.vae.safetensors"
        filename = get_filename(url)
        aria2_download(url=url, download_dir=vae_dir, filename=filename)
        print_line(80, color="green")
        auto_select_vae = True

    if auto_select_model:
        selected_model  = auto_select_file(ckpt_dir, "sd_model_checkpoint", ('.ckpt', '.safetensors'))
        cprint(f"Selected Model: {selected_model}", color="green")

    if auto_select_vae:
        selected_vae    = auto_select_file(vae_dir, "sd_vae", ('.vae.pt', '.vae.safetensors', '.pt', '.ckpt'))
        cprint(f"Selected VAE: {selected_vae}", color="green")

    # replacing faulty stuff
    with open(repo_dir / "requirements_versions.txt", "w") as f:
        f.write(requirements_file)
    #  /content/stable-diffusion-webui-forge/extensions/sd-hub
    !rm -rf {repo_dir / "extensions/sd-hub"}
    print_line(80, color="green")

    general_config_presets(config_file_path, lora_dir, use_presets, ui_config_file_path)

    if use_gradio_auth:
      cprint("Gradio Auth (use this account to login):", color="green")
      cprint("[-] Username: cagliostro", color="green")
      cprint("[-] Password:", password, color="green")
      print_line(80, color="green")

    config = {
        "enable-insecure-extension-access": True,
        "disable-safe-unpickle"           : True,
        "share"                           : True if not ngrok_token else False,
        "ngrok"                           : ngrok_token if ngrok_token else None,
        "ngrok-region"                    : ngrok_region if ngrok_token else None,
        "gradio-auth"                     : f"{user}:{password}" if use_gradio_auth else None,
        "no-hashing"                      : True,
        "disable-console-progressbars"    : True,
        "lowram"                          : True,
        "opt-sub-quad-attention"          : True,
        "opt-channelslast"                : True,
        "no-download-sd-model"            : True,
        "gradio-queue"                    : True,
        "listen"                          : True,
        "ckpt-dir"                        : ckpt_dir,
        "vae-dir"                         : vae_dir,
        "lora-dir"                        : lora_dir,
    }

    args = parse_args(config)
    final_args = f"python launch.py {args} {additional_arguments}"

    cprint()
    os.chdir(repo_dir)

if __name__ == "__main__":
    main()

In [ ]:
#@markdown ## (2) Run this cell before you start the webui (Only do it once), If you get a prompt to restart environment, just ignore it
!pip uninstall -y pydantic fastapi starlette gradio pydantic-settings typing-extensions || true
!pip install "pydantic>=2.1.1" "fastapi>=0.100.0" "starlette>=0.27.0" "pydantic-settings>=1.0.0" "gradio>=3.55.0" "typing-extensions>=4.5.0"
!pip install -U pillow

In [ ]:
#@markdown # (3) Start WebUI
#@markdown - Be sure to switch the value of "Diffusion in Low Bits", from "Automatic" to "float8-e5m2（fp16 LoRA)" (Only for SDXL not Flux)
! {final_args}


In [ ]:
#@markdown # run this to clear gpu vram and ram usage

import gc
import torch

def clear_gpu_memory():
  gc.collect()
  torch.cuda.empty_cache()

clear_gpu_memory()

# Optionally, you can also clear RAM usage using the following:
# import os
# os.system('reset') # This may or may not be effective depending on the environment



In [ ]:
# @title ## **Download Generated Images**
# @markdown Download file manually from files tab or save to Google Drive
import zipfile
from pathlib import Path
from google.colab import auth
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials
from colablib.colored_print import cprint

os.chdir(output_dir)

use_drive = False  # @param {type:"boolean"}
folder_name = "android-forge-colab"  # @param {type: "string"}
filename = "waifu.zip"  # @param {type: "string"}
save_as = filename

def get_unique_filename(base_filename):
    path = Path(base_filename)
    if not path.exists():
        return path
    i = 1
    while True:
        new_path = path.with_name(f"{path.stem}({i}){path.suffix}")
        if not new_path.exists():
            return new_path
        i += 1

filename = get_unique_filename(filename)

def zip_directory(directory, zipname):
    with zipfile.ZipFile(zipname, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for file_path in directory.rglob('*'):
            if file_path.is_file():
                zipf.write(file_path, file_path.relative_to(directory.parent))

zip_directory(output_dir, Path('/content/outputs.zip'))

if use_drive:
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive_service = GoogleDrive(gauth)

    def create_folder(folder_name):
        query = f"title='{folder_name}' and mimeType='application/vnd.google-apps.folder' and trashed=false"
        file_list = drive_service.ListFile({"q": query}).GetList()
        if file_list:
            cprint("Debug: Folder exists", color="green")
            return file_list[0]["id"]
        else:
            cprint("Debug: Creating folder", color="green")
            folder = drive_service.CreateFile({
                "title": folder_name,
                "mimeType": "application/vnd.google-apps.folder"
            })
            folder.Upload()
            return folder["id"]

    def upload_file(file_path, folder_id, save_as):
        save_as = get_unique_filename(save_as)
        file = drive_service.CreateFile({"title": save_as.name, "parents": [{"id": folder_id}]})
        file.SetContentFile(str(file_path))
        file.Upload()
        file.InsertPermission({"type": "anyone", "value": "anyone", "role": "reader"})
        return file["id"]

    folder_id = create_folder(folder_name)
    file_id = upload_file(Path('/content/outputs.zip'), folder_id, Path(save_as))
    sharing_link = f"https://drive.google.com/file/d/{file_id}/view?usp=sharing"
    cprint(f"Your sharing link: {sharing_link}", color="green")
else:
    cprint("Files zipped locally. Download manually from the files tab.", color="yellow")
